In [1]:
# %load ../import.py
import quandl
quandl.ApiConfig.api_key = 'MBMzvkxtv63KjFEV-tL6'
from quandl.errors.quandl_error import NotFoundError

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn import linear_model

import ipywidgets as widgets 
from ipywidgets import  interact, interactive, fixed, interact_manual
from IPython.display import display

#####################################################################################
# From tensorflow includes
import math
from matplotlib import cm
from matplotlib import gridspec
# from matplotlib import pyplot as plt

from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset


# For elongating day 
from datetime import timedelta

import csv



In [2]:
s_list=pd.read_csv("../sector_list.csv")

In [9]:
def select_sector(sector_name):
    
    result_file_name= "../Panel/Result/Predicted/"+sector_name+".csv"
    df= pd.read_csv(result_file_name, index_col="Date", parse_dates=["Date"])
    
    return df


sector_value=widgets.Dropdown(
                            options=s_list['sector'], 
                            description="Sector:"
                            )
w=widgets.interactive(select_sector, sector_name=sector_value)
display(w)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnU2VjdG9yOicsIG9wdGlvbnM9KCdBR1JPX0lOUFVUUycsICdBVVRPTU9CSUxFUycsICdBVVRPX0FOQ0lMTEFSSUXigKY=


In [24]:
df_predicted=w.result
df_predicted.apply(pd.to_numeric, errors="ignore")

,NSE/ARIES
Date,
2018-04-12,-0.011851
2018-04-13,0.005363
2018-04-14,-0.012235
2018-04-15,-0.012427
2018-04-16,-0.012619
2018-04-17,0.004886
2018-04-18,-0.013003


In [25]:
df_cols_list=list(df_predicted.columns)
idx_for_close= pd.date_range(start="2018-04-11", end="2018-04-18", freq="1d")
df_actual_close=pd.DataFrame(columns=df_cols_list, index=idx_for_close)
df_actual=pd.DataFrame(columns=df_cols_list, index=idx_for_close)
df_error=pd.DataFrame(columns=df_cols_list, index=df_predicted.index)
df_error_summary=pd.DataFrame(columns=df_cols_list, index=df_predicted.index)


In [26]:
for col in df_predicted.columns:
    data=quandl.get(col, start_date='2018-04-11', end_date='2018-04-18', column_index=5)
    df_actual_close[col]=data["Close"]
    print ("----Collecting data for:{0}".format(col))

----Collecting data for:NSE/ARIES


## Interpolating empty sets

In [27]:
df_actual_close.interpolate(method="spline", order=1,axis=0, limit_direction="both", inplace=True)
df_actual_close

,NSE/ARIES
2018-04-11,203.600000
2018-04-12,201.650000
2018-04-13,199.400000
2018-04-14,201.833589
2018-04-15,203.600614
2018-04-16,204.350000
2018-04-17,210.600000
2018-04-18,208.600000


## Actual Returns File 

In [28]:
for col in df_actual_close: 
    df_actual[col]=df_actual_close[[col]].apply (lambda x: (np.log(x/x.shift(1))) )
    
df_actual

,NSE/ARIES
2018-04-11,NaN
2018-04-12,-0.009624
2018-04-13,-0.011221
2018-04-14,0.012131
2018-04-15,0.008717
2018-04-16,0.003674
2018-04-17,0.030126
2018-04-18,-0.009542


## Error File

In [29]:
for col in df_error:
    df_error[col] = (df_actual[col]-df_predicted[col])
df_error

,NSE/ARIES
Date,
2018-04-12,0.002227
2018-04-13,-0.016584
2018-04-14,0.024366
2018-04-15,0.021144
2018-04-16,0.016293
2018-04-17,0.025240
2018-04-18,0.003461


## RMSE FILE

In [30]:
df_error_2=df_error.pow(2)
df_error_2

,NSE/ARIES
Date,
2018-04-12,0.000005
2018-04-13,0.000275
2018-04-14,0.000594
2018-04-15,0.000447
2018-04-16,0.000265
2018-04-17,0.000637
2018-04-18,0.000012


In [31]:
rmses={}
for col in df_error_2.columns:
    rmse=sum(df_error_2[col])/7
    rmses[col]=rmse
rmses

{'NSE/ARIES': 0.0003193214346420775}

## Summary File

In [33]:
df_error

,NSE/ARIES
Date,
2018-04-12,0.002227
2018-04-13,-0.016584
2018-04-14,0.024366
2018-04-15,0.021144
2018-04-16,0.016293
2018-04-17,0.025240
2018-04-18,0.003461


In [34]:
for col in df_error:
    df_error_summary[col] = 100*(df_error[col]/df_predicted[col])
    
df_error_summary

,NSE/ARIES
Date,
2018-04-12,-18.791399
2018-04-13,-309.222965
2018-04-14,-199.147273
2018-04-15,-170.143195
2018-04-16,-129.113748
2018-04-17,516.585789
2018-04-18,-26.619306


## Saving all the files

In [ ]:
sector_name=w.kwargs["sector_name"]
actual_file_name= "../Panel/Result/Actual/"+sector_name+".csv"
error_file_name= "../Panel/Result/Error/"+sector_name+".csv"
rmse_file_name="../Panel/Result/RMSE/"+sector_name+".csv"
summary_file_name="../Panel/Result/Summary/"+sector_name+".csv"


df_actual.to_csv(actual_file_name)
df_error.to_csv(error_file_name)
df_error_summary.describe().to_csv(summary_file_name)

#Saving RMSES
with open(rmse_file_name,'wb') as f:
    w = csv.writer(f)
    w.writerows(rmses.items())